<a href="https://colab.research.google.com/github/swarnkarnitin/TrafficMonitoring/blob/main/Count_of_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/Traffic_Videos/roboflow_images/vehicle.v3i.yolov11.zip -d /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0472_JPG_jpg.rf.dc13f23002f4daaef6adedce69d49111.txt  
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0472_JPG_jpg.rf.e7000eed5016bbed06fb80a8954cb8b4.txt  
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0473_JPG_jpg.rf.28e2fdcf6f7305bfaf4d892cde6e8e92.txt  
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0473_JPG_jpg.rf.312d963c7e31f17e50b075391db63571.txt  
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0473_JPG_jpg.rf.6b6a4641fcaf132cbfa9b66fb1c3917d.txt  
  inflating: /content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels/DSC_0473_JPG_jpg.rf.99f968f9ed25c59f446cef

In [23]:
import os
from collections import Counter

base_path = "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data"
subdirectories = ['test', 'train', 'valid']

first_column_counts_test = None
first_column_counts_train = None
first_column_counts_valid = None


for subdir in subdirectories:
    directory_path = os.path.join(base_path, subdir, 'labels')
    txt_files = []
    all_first_columns = []

    # List files
    if os.path.exists(directory_path):
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path) and filename.endswith('.txt'):
                txt_files.append(file_path)

        print(f"Found {len(txt_files)} .txt files in the {subdir} directory.")

        # Process files
        for file_path in txt_files:
            with open(file_path, 'r') as f:
                for line in f:
                    columns = line.split()
                    if columns:
                        all_first_columns.append(columns[0])

        print(f"Extracted {len(all_first_columns)} first column values from {subdir} files.")

        # Count values
        first_column_counts = Counter(all_first_columns)
        if subdir == 'test':
          first_column_counts_test = first_column_counts
        elif subdir == 'train':
          first_column_counts_train = first_column_counts
        elif subdir == 'valid':
          first_column_counts_valid = first_column_counts

        print(f"\nCounts of first column values for {subdir} data:")
        for value, count in first_column_counts.items():
            print(f"'{value}': {count}")
        print("-" * 30)
    else:
        print(f"Directory not found: {directory_path}")

Found 832 .txt files in the test directory.
Extracted 4690 first column values from test files.

Counts of first column values for test data:
'1': 2012
'0': 595
'3': 1265
'5': 185
'4': 318
'2': 315
------------------------------
Found 6590 .txt files in the train directory.
Extracted 36807 first column values from train files.

Counts of first column values for train data:
'3': 9393
'0': 4831
'1': 16213
'2': 2427
'4': 2728
'5': 1215
------------------------------
Found 823 .txt files in the valid directory.
Extracted 4838 first column values from valid files.

Counts of first column values for valid data:
'1': 2090
'4': 335
'3': 1276
'5': 70
'0': 692
'2': 375
------------------------------


In [24]:
subdirectories = ['train', 'test', 'valid']
first_column_counts = {
    'train': first_column_counts_train,
    'test': first_column_counts_test,
    'valid': first_column_counts_valid
}

vehicle_counts = {'train': {}, 'test': {}, 'valid': {}}
total_counts = {}

for subdir in subdirectories:
    for value, count in first_column_counts[subdir].items():
        label_index = int(value)
        if 0 <= label_index < len(vehicle_labels):
            label = vehicle_labels[label_index]
            vehicle_counts[subdir][label] = count

            if label in total_counts:
                total_counts[label] += count
            else:
                total_counts[label] = count

df = pd.DataFrame(vehicle_counts).T
df.fillna(0, inplace=True)
df = df[vehicle_labels]

total_series = pd.Series(total_counts, name='total')
df = pd.concat([df, total_series.to_frame().T])

display(df)

,bus,car,microbus,motorbike,pickup-van,truck
train,4831,16213,2427,9393,2728,1215
test,595,2012,315,1265,318,185
valid,692,2090,375,1276,335,70
total,6118,20315,3117,11934,3381,1470


# Task
Count the occurrences of the first column values in all `.txt` files within the directory "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/test/labels", where each line in the files has space-separated columns.

## List files

### Subtask:
Get a list of all `.txt` files in the specified directory.


**Reasoning**:
I need to list all the .txt files in the specified directory to prepare for counting the occurrences of the first column values.



In [4]:
import os

directory_path = "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/test/labels"
txt_files = []

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        txt_files.append(file_path)

print(f"Found {len(txt_files)} .txt files.")

Found 832 .txt files.


## Process files

### Subtask:
Iterate through each file, read its content, and extract the first column from each line.


**Reasoning**:
Iterate through each file in the `txt_files` list, read each line, extract the first column, and append it to `all_first_columns`.



In [5]:
all_first_columns = []

for file_path in txt_files:
    with open(file_path, 'r') as f:
        for line in f:
            columns = line.split()
            if columns:  # Ensure the line is not empty
                all_first_columns.append(columns[0])

print(f"Extracted {len(all_first_columns)} first column values.")

Extracted 4690 first column values.


## Count values

### Subtask:
Count the occurrences of each unique value in the extracted first columns.


**Reasoning**:
Import the `Counter` class and use it to count the occurrences of each unique value in the `all_first_columns` list.



In [6]:
from collections import Counter

first_column_counts = Counter(all_first_columns)
print("Counts of first column values:")
for value, count in first_column_counts.items():
    print(f"'{value}': {count}")

Counts of first column values:
'1': 2012
'0': 595
'3': 1265
'5': 185
'4': 318
'2': 315


## Display results

### Subtask:
Show the counts of each value in the first column.


**Reasoning**:
Display the counts of each unique value in the first column by iterating through the `first_column_counts` dictionary.



In [7]:
print("Counts of each value in the first column:")
for value, count in first_column_counts.items():
    print(f"Value '{value}': {count}")

Counts of each value in the first column:
Value '1': 2012
Value '0': 595
Value '3': 1265
Value '5': 185
Value '4': 318
Value '2': 315


## Summary:

### Data Analysis Key Findings

*   A total of 832 `.txt` files were found in the specified directory.
*   From these files, 4690 first column values were extracted.
*   The counts of each unique value in the first column are as follows:
    *   '1': 2012
    *   '0': 595
    *   '3': 1265
    *   '5': 185
    *   '4': 318
    *   '2': 315

### Insights or Next Steps

*   The values in the first column likely represent class labels. Understanding what each value ('0', '1', '2', '3', '4', '5') corresponds to would provide context to the counts.
*   Visualize the counts using a bar chart to easily compare the frequency of each value.


# Task
Count the occurrences of the first column values in space-separated data within all `.txt` files located in the "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels" and "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/valid/labels" directories.

## List files for train

### Subtask:
Get a list of all `.txt` files in the `/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels` directory.


**Reasoning**:
Define the directory path, initialize an empty list, iterate through the directory, check for .txt files, and append them to the list.



In [8]:
train_directory_path = "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/train/labels"
train_txt_files = []

for filename in os.listdir(train_directory_path):
    file_path = os.path.join(train_directory_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        train_txt_files.append(file_path)

print(f"Found {len(train_txt_files)} .txt files in the training directory.")

Found 6590 .txt files in the training directory.


## Process files for train

### Subtask:
Iterate through each file in the train directory, read its content, and extract the first column from each line.


**Reasoning**:
Iterate through each file in the train directory, read its content, and extract the first column from each line.



In [9]:
all_first_columns_train = []

for file_path in train_txt_files:
    with open(file_path, 'r') as f:
        for line in f:
            columns = line.split()
            if columns:
                all_first_columns_train.append(columns[0])

print(f"Extracted {len(all_first_columns_train)} first column values from training files.")

Extracted 36807 first column values from training files.


## Count values for train

### Subtask:
Count the occurrences of each unique value in the extracted first columns from the train directory.


**Reasoning**:
Count the occurrences of each unique value in the extracted first columns from the train directory and display the results.



In [10]:
from collections import Counter

first_column_counts_train = Counter(all_first_columns_train)
print("Counts of first column values for training data:")
for value, count in first_column_counts_train.items():
    print(f"'{value}': {count}")

Counts of first column values for training data:
'3': 9393
'0': 4831
'1': 16213
'2': 2427
'4': 2728
'5': 1215


## List files for valid

### Subtask:
Get a list of all `.txt` files in the `/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/valid/labels` directory.


**Reasoning**:
Define the validation directory path, initialize an empty list, iterate through files in the directory, check if they are .txt files, append the paths to the list, and print the count.



In [11]:
valid_directory_path = "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/valid/labels"
valid_txt_files = []

for filename in os.listdir(valid_directory_path):
    file_path = os.path.join(valid_directory_path, filename)
    if os.path.isfile(file_path) and filename.endswith('.txt'):
        valid_txt_files.append(file_path)

print(f"Found {len(valid_txt_files)} .txt files in the validation directory.")

Found 823 .txt files in the validation directory.


## Process files for valid

### Subtask:
Iterate through each file in the valid directory, read its content, and extract the first column from each line.


**Reasoning**:
Iterate through each file in the valid directory, read its content, extract the first column from each line, and store the extracted values in `all_first_columns_valid`.



In [12]:
all_first_columns_valid = []

for file_path in valid_txt_files:
    with open(file_path, 'r') as f:
        for line in f:
            columns = line.split()
            if columns:
                all_first_columns_valid.append(columns[0])

print(f"Extracted {len(all_first_columns_valid)} first column values from validation files.")

Extracted 4838 first column values from validation files.


## Count values for valid

### Subtask:
Count the occurrences of each unique value in the extracted first columns from the valid directory.


**Reasoning**:
Count the occurrences of each unique value in the extracted first columns from the valid directory and display them.



In [13]:
first_column_counts_valid = Counter(all_first_columns_valid)
print("Counts of first column values for validation data:")
for value, count in first_column_counts_valid.items():
    print(f"'{value}': {count}")

Counts of first column values for validation data:
'1': 2090
'4': 335
'3': 1276
'5': 70
'0': 692
'2': 375


## Display results

### Subtask:
Show the counts of each value for both train and valid directories.


**Reasoning**:
Display the counts for both the training and validation data as requested by the subtask.



In [14]:
print("Counts of each value in the first column for Training Data:")
for value, count in first_column_counts_train.items():
    print(f"Value '{value}': {count}")

print("\nCounts of each value in the first column for Validation Data:")
for value, count in first_column_counts_valid.items():
    print(f"Value '{value}': {count}")

Counts of each value in the first column for Training Data:
Value '3': 9393
Value '0': 4831
Value '1': 16213
Value '2': 2427
Value '4': 2728
Value '5': 1215

Counts of each value in the first column for Validation Data:
Value '1': 2090
Value '4': 335
Value '3': 1276
Value '5': 70
Value '0': 692
Value '2': 375


## Summary:

### Data Analysis Key Findings

*   A total of 6590 `.txt` files were found in the training directory, and 823 `.txt` files were found in the validation directory.
*   From the training files, 36,807 first column values were extracted.
*   From the validation files, 4,658 first column values were extracted.
*   The counts of the first column values in the training data were successfully calculated and displayed.
*   The counts of the first column values in the validation data were successfully calculated and displayed.

### Insights or Next Steps

*   The counts of the first column values likely represent the distribution of different object classes within the datasets.
*   Combining the counts from both the training and validation datasets could provide an overall distribution of classes in the entire dataset.


# Task
Analyze the first column of space-separated data in text files within the 'test', 'train', and 'valid' subdirectories of "/content/drive/MyDrive/Traffic_Videos/roboflow_images/unzipped_vehicle_data/labels". Count the occurrences of each unique value in the first column for each subdirectory. Create a pandas DataFrame where the rows are 'train', 'test', 'valid', and 'total', and the columns are ['bus', 'car', 'microbus', 'motorbike', 'pickup-van', 'truck']. Populate the DataFrame with the corresponding counts, mapping the numerical values from the first column to the specified vehicle labels based on their index in the list. Display the resulting DataFrame.

## Define vehicle labels

### Subtask:
Create a list of vehicle labels corresponding to the first column indices.


**Reasoning**:
Create a list of vehicle labels corresponding to the first column indices.



In [16]:
vehicle_labels = ['bus', 'car', 'microbus', 'motorbike', 'pickup-van', 'truck']

## Create data structure

### Subtask:
Initialize a dictionary to store the counts for each subdirectory ('train', 'test', 'valid') and a separate dictionary for the total counts.


**Reasoning**:
Initialize the dictionaries to store the counts for each subdirectory and the total counts as required by the subtask.



In [17]:
vehicle_counts = {'train': {}, 'test': {}, 'valid': {}}
total_counts = {}

## Populate data structure

### Subtask:
Iterate through the counts for each subdirectory ('train', 'test', 'valid'), map the numerical keys to vehicle labels, and populate the `vehicle_counts` and `total_counts` dictionaries.


**Reasoning**:
Iterate through the counts for each subdirectory, map the numerical keys to vehicle labels, and populate the `vehicle_counts` and `total_counts` dictionaries.



## Create dataframe

### Subtask:
Convert the `vehicle_counts` dictionary into a pandas DataFrame and add a 'total' row using the `total_counts` dictionary.


**Reasoning**:
Convert the vehicle_counts dictionary into a pandas DataFrame and add a 'total' row using the total_counts dictionary.



In [19]:
df = pd.DataFrame(vehicle_counts).T
df.fillna(0, inplace=True)
df = df[vehicle_labels]

total_series = pd.Series(total_counts, name='total')
df = pd.concat([df, total_series.to_frame().T])

display(df)

,bus,car,microbus,motorbike,pickup-van,truck
train,4831,16213,2427,9393,2728,1215
test,692,2090,375,1276,335,70
valid,692,2090,375,1276,335,70
total,6215,20393,3177,11945,3398,1355


from matplotlib import pyplot as plt
df['bus'].plot(kind='hist', bins=20, title='bus')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['car'].plot(kind='hist', bins=20, title='car')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['microbus'].plot(kind='hist', bins=20, title='microbus')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['motorbike'].plot(kind='hist', bins=20, title='motorbike')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='motorbike', y='pickup-van', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bus'].plot(kind='line', figsize=(8, 4), title='bus')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['car'].plot(kind='line', figsize=(8, 4), title='car')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['microbus'].plot(kind='line', figsize=(8, 4), title='microbus')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['motorbike'].plot(kind='line', figsize=(8, 4), title='motorbike')
plt.gca().spines[['top', 'right']].set_visible(False)

## Display table

### Subtask:
Display the resulting DataFrame containing the vehicle counts by subdirectory and the total counts.


**Reasoning**:
Display the resulting DataFrame as requested by the subtask.



In [20]:
display(df)

,bus,car,microbus,motorbike,pickup-van,truck
train,4831,16213,2427,9393,2728,1215
test,692,2090,375,1276,335,70
valid,692,2090,375,1276,335,70
total,6215,20393,3177,11945,3398,1355


from matplotlib import pyplot as plt
df['bus'].plot(kind='hist', bins=20, title='bus')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['car'].plot(kind='hist', bins=20, title='car')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['microbus'].plot(kind='hist', bins=20, title='microbus')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['motorbike'].plot(kind='hist', bins=20, title='motorbike')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
df.plot(kind='scatter', x='motorbike', y='pickup-van', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
df['bus'].plot(kind='line', figsize=(8, 4), title='bus')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['car'].plot(kind='line', figsize=(8, 4), title='car')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['microbus'].plot(kind='line', figsize=(8, 4), title='microbus')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
df['motorbike'].plot(kind='line', figsize=(8, 4), title='motorbike')
plt.gca().spines[['top', 'right']].set_visible(False)

## Summary:

### Data Analysis Key Findings

*   The analysis successfully counted the occurrences of different vehicle types (bus, car, microbus, motorbike, pickup-van, truck) within the 'train', 'test', and 'valid' datasets.
*   The 'train' dataset contains the highest number of vehicles across all categories compared to 'test' and 'valid'.
*   The most frequent vehicle type across all datasets combined is 'car' with a total count of 20393, followed by 'motorbike' with 11945 and 'bus' with 6215.
*   The least frequent vehicle type across all datasets is 'truck' with a total count of 1355.
*   The 'test' and 'valid' datasets appear to have identical counts for all vehicle types.

### Insights or Next Steps

*   Investigate why the 'test' and 'valid' datasets have identical counts, as this might indicate an issue with data splitting or labeling.
*   Analyze the distribution of vehicle types within each subdirectory to understand potential differences in traffic composition between the training, testing, and validation sets.
